In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from IPython.display import clear_output

from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### READ THE TRAIN AND TEST DATASET

In [ ]:
test_fp = '/kaggle/input/ncair-2023-ai-competition/test.csv' 
train_fp = '/kaggle/input/ncair-2023-ai-competition/train.csv'

#LAODING THE DATASET FOR TRAINING AND TESTING
df_train = pd.read_csv(train_fp,index_col='BVN') # training data
df_test = pd.read_csv(test_fp,index_col='BVN') # testing data

### **Step 2:** Check the general info about your data

- This will also tell you the number of data entries you have

- Use the information to note down the categorical columns (features), you might need it later

In [ ]:
df_train.info()

### CHECK THE STATS OF THE DATASET

In [ ]:
df_train.describe()

### WORK ON EDUCTION MISSING VALUE

In [ ]:
# Calculate the mode of the column
mode_value = df_train["EDUCATION"].mode()[0]
# Replace missing values with the mode
df_train['EDUCATION'].fillna(mode_value, inplace=True)

# Calculate the mode of the column
mode_value = df_test["EDUCATION"].mode()[0]
# Replace missing values with the mode
df_train['EDUCATION'].fillna(mode_value, inplace=True)


### CHECK FOR  ZEROS IN CATEGORICAL COLUMNS AND REPLACE WITH THE MODE

In [ ]:
#CHECK FOR ZEROS IN CATEGORICAL COLUMNS
df_train['MARRIAGE'].value_counts()

In [ ]:
# List of categorical columns
categorical_columns = ['SEX', 'MARRIAGE', 'EDUCATION', 'Nationality']

# Iterate over the categorical columns
for col in categorical_columns:
    # Find rows with zero values
    zero_rows = df_train[col] == 0
    zero_rows_t = df_test[col] == 0
    
    # Calculate the mode of the column
    mode_value = df_train[col].mode()[0]
    mode_value_t = df_test[col].mode()[0]
    
    
    # Replace zero values with the mode
    df_train.loc[zero_rows, col] = mode_value
    df_test.loc[zero_rows_t, col] = mode_value_t



# Print the updated DataFrame
df_train.head()

In [ ]:
mode = df_train['MARRIAGE'].mode()[0]
df_train['MARRIAGE'] = df_train['MARRIAGE'].replace('0', mode)

mode = df_test['MARRIAGE'].mode()[0]
df_test['MARRIAGE'] = df_test['MARRIAGE'].replace('0', mode)

### CHECK FOR ZEROS IN CATEGORICAL DATA

In [ ]:
f= ['SEX', 'MARRIAGE', 'EDUCATION', 'Nationality']
df_train.index.value_counts()

In [ ]:
df_train.isna().sum()

### HANDLING THE DOB COLUMN

In [ ]:
from datetime import datetime

current_year = datetime.now().year
df_train['DOB'] = pd.to_datetime(df_train['DOB'], format='%m/%d/%Y', errors='coerce')
df_train['Age'] = current_year - df_train['DOB'].dt.year

mean = df_train['Age'].mean()
df_train['Age'].fillna(mean, inplace=True)
df_train['Age'].fillna(0, inplace=True)  # Fill remaining NaN values with 0

df_train.drop('DOB', axis=1, inplace=True)


#############################TEST##################################
from datetime import datetime

current_year = datetime.now().year
df_test['DOB'] = pd.to_datetime(df_test['DOB'], format='%m/%d/%Y',errors='coerce')
df_test['Age'] = current_year - df_test['DOB'].dt.year

mean = df_test['Age'].mean()
df_test['Age'].fillna(mean, inplace=True)
df_test['Age'].fillna(0, inplace=True)  # Fill remaining NaN values with 0

df_test.drop('DOB', axis=1, inplace=True)




### CHECKING NORMAL DISTRIBUTION

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a distribution plot of all columns
fig, axes = plt.subplots(nrows=len(df_train.columns), figsize=(10, 50))
for i, column in enumerate(df_train.columns):
    sns.histplot(data=df_train[column], ax=axes[i], kde=True)
    axes[i].set_title(column)
    axes[i].set_xlabel('')

plt.tight_layout()
plt.show()

### SUMING TO CHECK IF SIMILAR INFO EXISTS

In [ ]:
df_train['Total_Bill_Amount'] = df_train[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].sum(axis=1)
df_train['Total_PAY'] = df_train[['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].sum(axis=1)
df_train['Total_PAY_AMT'] = df_train[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].sum(axis=1)
df_train['Average_PAY'] = df_train['Total_PAY']/6
df_train['Average_Bill_Amount']=df_train['Total_Bill_Amount'] /6
df_train['Average_PAY_AMT']=df_train['Total_PAY_AMT'] /6



#TEST
df_test['Total_Bill_Amount'] = df_test[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].sum(axis=1)
df_test['Total_PAY'] = df_test[['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].sum(axis=1)
df_test['Total_PAY_AMT'] = df_test[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].sum(axis=1)
df_test['Average_PAY'] = df_test['Total_PAY']/6
df_test['Average_Bill_Amount']=df_test['Total_Bill_Amount'] /6
df_test['Average_PAY_AMT']=df_test['Total_PAY_AMT'] /6

### HANDLING SKEWED COLUMNS

In [ ]:


#USING LOG TRANSFORMS
# Define the columns to transform
columns_to_transform = ['LIMIT_BAL','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

# Create a copy of the original DataFrame
transformed_data = df_train.copy()

# Apply log transformation to each column
for column in columns_to_transform:
    transformed_column = np.log1p(df_train[column])
    transformed_data[column] = transformed_column

# Replace the original columns with the transformed columns in the original DataFrame
df_train[columns_to_transform] = transformed_data[columns_to_transform]



##################TEST###############
#USING LOG TRANSFORMS
# Define the columns to transform
columns_to_transform = ['LIMIT_BAL','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

# Create a copy of the original DataFrame
transformed_data = df_test.copy()

# Apply log transformation to each column
for column in columns_to_transform:
    transformed_column = np.log1p(df_test[column])
    transformed_data[column] = transformed_column

# Replace the original columns with the transformed columns in the original DataFrame
df_test[columns_to_transform] = transformed_data[columns_to_transform]



#df_train['Total_Bill_Amount'] = df_train[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].sum(axis=1)

# import scipy.stats as stats

# # Shift the values to make them positive
# shift_constant = abs(df_train[['Total_PAY_AMT', 'Total_Bill_Amount']].min().min()) + 1  # Add 1 to avoid zero values
# shifted_total_pay_amt = df_train['Total_PAY_AMT'] + shift_constant
# shifted_total_bill_amt = df_train['Total_Bill_Amount'] + shift_constant

# # Apply Box-Cox transformation
# transformed_total_pay_amt, _ = stats.boxcox(shifted_total_pay_amt)
# transformed_total_bill_amt, _ = stats.boxcox(shifted_total_bill_amt)

# # Replace the original columns with the transformed columns in the DataFrame
# df_train['Total_PAY_AMT'] = transformed_total_pay_amt
# df_train['Total_Bill_Amount'] = transformed_total_bill_amt

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a distribution plot of all columns
fig, axes = plt.subplots(nrows=len(df_train.columns), figsize=(10, 50))
for i, column in enumerate(df_train.columns):
    sns.histplot(data=df_train[column], ax=axes[i], kde=True)
    axes[i].set_title(column)
    axes[i].set_xlabel('')

plt.tight_layout()
plt.show()

In [ ]:
df_train.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

# Create a box plot of all columns
df_train.boxplot(figsize=(12, 8))
plt.title('Box Plot of Columns')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_train.describe()

### ENCODE CATEGORICAL DATA

In [ ]:
categorical_columns = ['SEX', 'MARRIAGE', 'EDUCATION', 'Nationality']

list(df_train.EDUCATION.unique())

In [ ]:
# PERFORM ONEHOT FOR SEX
# Perform one-hot encoding
encoded_df = pd.get_dummies(df_train['SEX'], prefix='SEX')

# Concatenate the encoded columns with the original DataFrame
df_train = pd.concat([df_train, encoded_df], axis=1)

# Drop the original 'SEX' column
df_train.drop('SEX', axis=1, inplace=True)




#PERFORM ORDINAL ENCODING TO MARRIAGE
import pandas as pd
# Define the mapping of categories to numerical values
mapping = {'Single': 0, 'Divorce': 1, 'Married': 2}
# Perform ordinal encoding
df_train['MARRIAGE'] = df_train['MARRIAGE'].replace(mapping)

##############################################################################

#PERFORM ORDINAL ENCODING TO EDUCATION
education_mapping = {
    'Uneducated': 0,
    'Undergraduate': 1,
    'Graduate': 2,
    'Masters': 3,
    'Doctorate': 4
}

# Perform ordinal encoding
df_train['EDUCATION'] = df_train['EDUCATION'].replace(education_mapping)

#####################################################################################


#PERFORM ORDINAL ENCODING TO  NATIONALITY
nationality_mapping={'Nigeria':0}

df_train['Nationality'] = df_train['Nationality'].replace(nationality_mapping)






################TEST


# PERFORM ONEHOT FOR SEX
# Perform one-hot encoding
encoded_df = pd.get_dummies(df_test['SEX'], prefix='SEX')

# Concatenate the encoded columns with the original DataFrame
df_test = pd.concat([df_test, encoded_df], axis=1)

# Drop the original 'SEX' column
df_test.drop('SEX', axis=1, inplace=True)



#PERFORM ORDINAL ENCODING TO MARRIAGE
import pandas as pd
# Define the mapping of categories to numerical values
mapping = {'Single': 0, 'Divorce': 1, 'Married': 2}
# Perform ordinal encoding
df_test['MARRIAGE'] = df_test['MARRIAGE'].replace(mapping)

##############################################################################

#PERFORM ORDINAL ENCODING TO EDUCATION
education_mapping = {
    'Uneducated': 0,
    'Undergraduate': 1,
    'Graduate': 2,
    'Masters': 3,
    'Doctorate': 4
}

# Perform ordinal encoding
df_test['EDUCATION'] = df_test['EDUCATION'].replace(education_mapping)

#####################################################################################


#PERFORM ORDINAL ENCODING TO  NATIONALITY
nationality_mapping={'Nigeria':0}

df_test['Nationality'] = df_test['Nationality'].replace(nationality_mapping)



In [ ]:
df_train.head()

### CHECKING FOR MUTUAL INFORMATION

In [ ]:
from sklearn.feature_selection import mutual_info_regression

X = df_train.drop('default', axis=1)  # Drop the target variable from the feature matrix
y = df_train['default']  # Target variable

def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y)

In [ ]:
mi_scores

In [ ]:
# Assuming df_train contains your dataset
columns_to_drop = ['Nationality','SEX_Male','SEX_Female','MARRIAGE']

# Drop the specified columns from the dataframe
df_train.drop(columns_to_drop, axis=1, inplace=True)
df_test.drop(columns_to_drop, axis=1, inplace=True)

### USING K MEANS TO CLUSTER

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import seaborn as sns

# Assuming df_train contains your dataset with the features mentioned
features = ['Total_PAY', 'PAY_1', 'PAY_5', 'PAY_3', 'PAY_2', 'PAY_4', 'PAY_AMT3', 'PAY_6', 'Total_PAY_AMT', 'PAY_AMT1',
            'PAY_AMT2', 'PAY_AMT6', 'BILL_AMT5', 'PAY_AMT4', 'PAY_AMT5', 'BILL_AMT1', 'Total_Bill_Amount', 'BILL_AMT6',
            'BILL_AMT4', 'LIMIT_BAL', 'BILL_AMT2', 'BILL_AMT3']

# Create a subset of the dataframe with the selected features
X = df_train[features]
G = df_test[features]

# Specify the number of clusters (K)
num_clusters = 5

# Create an instance of the KMeans class
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans_t = KMeans(n_clusters=num_clusters, random_state=0)

# Fit the K-means model to the data
kmeans.fit(X)
kmeans_t.fit(G)

# Get the cluster labels assigned to each data point
cluster_labels = kmeans.labels_
cluster_labels_t = kmeans_t.labels_

# Add the cluster labels to the original dataframe
df_train['Cluster'] = cluster_labels
df_test['Cluster'] = cluster_labels_t

# Create a scatter plot using relplot
sns.relplot(data=df_train, x='Total_PAY', y='PAY_1', hue='Cluster')
plt.xlabel('Total_PAY')
plt.ylabel('PAY_1')
plt.title('K-means Clustering')
plt.show()

### SPLITTING AND STANDARDIZATION

In [ ]:
from sklearn.model_selection import train_test_split
#SPLIT INTO X AND Y
y=df_train['default']
x=df_train.drop(columns=['default'],axis=1,inplace=True)

In [ ]:
len(list(df_train.columns))
len(list(df_test.columns))

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


# Exclude the 'index' column from the DataFrame
df_train_without_index = df_train.reset_index(drop=True)
BVN=df_test.index
df_test_without_index = df_test.reset_index(drop=True)

# Apply SimpleImputer to handle missing values
imputer = SimpleImputer()
imputed_data = imputer.fit_transform(df_train_without_index)
imputed_data_t = imputer.fit_transform(df_test_without_index)



# Apply StandardScaler to the imputed data
scaler = StandardScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(imputed_data), columns=df_train_without_index.columns)
x_scaled_t = pd.DataFrame(scaler.fit_transform(imputed_data_t), columns=df_test_without_index.columns)

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.utils import check_array

# Assuming you have your features (X) and target variable (y) DataFrame

# Validate and reshape input data
X = check_array(x_scaled, ensure_2d=True)
y = check_array(y, ensure_2d=False)

# Create an instance of SMOTE
smote = SMOTE()

# Perform oversampling on the dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert the resampled arrays back to DataFrames
X_resampled = pd.DataFrame(X_resampled, columns=df_train_without_index.columns)
y_resampled = pd.DataFrame(y_resampled, columns=['default'])

# Check the class distribution after oversampling
class_counts = y_resampled['default'].value_counts()
print("Class Distribution after Oversampling:\n", class_counts)


In [ ]:
X_resampled.shape

### CHECK FOR COLLINEARITY IN THE STANDARDIZED DATA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = x_scaled.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.show()

correlation_matrix

### DEALING WITH MULTICOLLINEARITY

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Calculate the correlation matrix
# correlation_matrix = df_scaled_reconstructed.corr()

# # Plot the heatmap
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix Heatmap')
# plt.show()

In [ ]:
 #y.value_counts()

In [ ]:
# Combine x_scaled and y into a single DataFrame
df_scaled = X_resampled
df_tests = x_scaled_t
df_scaled.head()

In [ ]:
y_resampled.reset_index(drop=True,inplace=True)
x_scaled_t.reset_index(drop=True,inplace=True)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df_scaled,y_resampled, test_size=0.2,random_state=50)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import Objective, BayesianOptimization

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_1', min_value=0, max_value=20, step=2), activation='relu', input_shape=(28,)))
    model.add(layers.Dense(units=hp.Int('units_2', min_value=0, max_value=20, step=2), activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy', tf.keras.metrics.AUC()])

    return model

def search_hyperparameters(x_train, y_train, x_val, y_val, max_trials=20, epochs=10):
    objective = Objective('val_auc', direction='max')
    tuner = BayesianOptimization(
        build_model,
        objective=objective,
        max_trials=max_trials,
        executions_per_trial=1,
        directory='hyperparameter_search_2jII',
        project_name='model_tuning'
    )

    tuner.search(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val))

    best_model = tuner.get_best_models(num_models=1)[0]
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

    return best_model, best_hyperparameters

# Perform hyperparameter search
best_model, best_hyperparameters = search_hyperparameters(x_train, y_train, x_val, y_val)

In [ ]:
best_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow import keras




# Create the MLP model
model = keras.Sequential([
    keras.layers.Dense(20, activation='relu',input_shape=[28]),
    #keras.layers.BatchNormalization(),
    #keras.layers.Dropout(0.3),
    keras.layers.Dense(20, activation='relu'),
    #keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=[ tf.keras.metrics.AUC()])


model.summary()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history=model.fit(x_train, y_train, 
          batch_size=12, 
          epochs=20,
          callbacks=[early_stopping],
          shuffle=True,
          validation_data=(x_val, y_val))

In [ ]:
# Plot the training and validation losses
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import pandas as pd

# Check missing columns
missing_columns = set(df_tests.columns).symmetric_difference(set(x_train.columns))

if len(missing_columns) > 0:
    print("Missing columns:")
    for column in missing_columns:
        if column in df_tests.columns and column not in x_train.columns:
            print(f"- Column '{column}' is missing in x_train.")
        elif column in x_train.columns and column not in df_tests.columns:
            print(f"- Column '{column}' is missing in df_tests.")
else:
    print("No missing columns found.")


In [ ]:
df_test.columns

In [ ]:
import matplotlib.pyplot as plt


# Access the accuracy and loss values from the history object
accuracy = history.history[tf.keras.metrics.AUC()]
val_accuracy = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the accuracy curve
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the loss curve
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### XGBCLASSIFIER

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.4, 0.6, 0.8]
}

# Create an XGBoost classifier object
model = xgb.XGBClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(x_scaled, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best AUC Score:", grid_search.best_score_)

In [ ]:
model_2 = xgb.XGBClassifier(colsample_bytree=0.4,
                            learning_rate=0.01,
                            max_depth=6,
                            n_estimators=500,
                            subsample=0.6,
                            eval_metric='auc')

In [ ]:
model_2.fit(x_scaled, y)

In [ ]:
# Assuming you have a test dataset named `x_test` for prediction

# Perform predictions
predictions = model_2.predict(df_tests)

# Print the predicted values
print(predictions)


# Create a DataFrame with columns "BVN" and "default" and the predicted values
predictions_df = pd.DataFrame({'BVN': BVN, 'default': predictions})

# Print the DataFrame
print(predictions_df)


# Set the "BVN" column as the index
predictions_df.set_index('BVN', inplace=True)

# Print the updated DataFrame
print(predictions_df)



In [ ]:
import matplotlib.pyplot as plt

# Get the training history from the XGBoost model
train_loss = model_2.evals_result()['validation_0']['logloss']
train_accuracy = model_2.evals_result()['validation_0']['accuracy']
eval_loss = model_2.evals_result()['validation_1']['logloss']
eval_accuracy = model_2.evals_result()['validation_1']['accuracy']

# Create a list of epochs based on the length of the training history
epochs = range(1, len(train_loss) + 1)

# Plot the training loss and accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, eval_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, eval_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

### LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

# Assuming X_train and y_train are your training features and labels
# Assuming X_test and y_test are your testing features and labels

# Create an instance of LogisticRegression
logreg = LogisticRegression()

# Train the logistic regression model
logreg.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Make predictions on the testing dataset
y_pred = logreg.predict(x_val)
x_val, y_val
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_valy_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("ROC-AUC:", roc_auc)


In [ ]:
predictions = model.predict(df_tests)


In [ ]:
def make_predictions(model, test_data):
    # Get the column to use for prediction
    test_column = test_data
    
    # Use the trained model to predict y_train based on the test column
    predictions = model.predict(test_column)
    
    return predictions


predictions = make_predictions(model, df_tests)
print(predictions)

p=predictions

prediction = np.where(p >= 0.5, 1, 0)


In [ ]:
pf = pd.DataFrame(prediction,df_tests.index)
pf.value_counts()

In [ ]:
import pandas as pd

# Assuming you have a DataFrame called df with existing column names
# Define a dictionary mapping old column names to new column names
column_names = {
    0: 'default',
}

# Rename the columns using the dictionary
pf = pf.rename(columns=column_names)


In [ ]:
pf['BVN']=BVN

In [ ]:
pf.set_index('BVN', inplace=True)

In [ ]:
pf

In [ ]:
pf.to_csv('/kaggle/working/prediction_June.csv')

In [ ]:
def make_predictions(model, test_data, column_name):
    # Get the column to use for prediction
    test_column = test_data[column_name]
    
    # Use the trained model to predict y_train based on the test column
    predictions = model.predict(test_column)
    
    return predictions

predictions = make_predictions(model, df_tests, 'BVN')
pf = pd.DataFrame(predictions,df_tests['BVN'])